In [65]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # or "0,1"

In [ ]:
wandb.init(project="tesi", name="bigbird")  # args = TrainingArguments(
wandb.stop()

In [1]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from datasets import Dataset

tokenizer = AutoTokenizer.from_pretrained("./robertaLargeIofNil/checkpoint-2178")
model = AutoModelForQuestionAnswering.from_pretrained(
    "./robertaLargeIofNil/checkpoint-2178", return_dict=False
).to("cuda:0")

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
import json
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenized = []
file_path = "./nil_el_instanceof.jsonl"
from datasets import Dataset

original_data = []
with open(file_path, "r") as file:
    for line in file:
        # Load each line as a JSON object
        data_line = json.loads(line)
        
        
        original_data.append(data_line)
print(len(original_data))

7939


In [4]:
import json
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenized = []
file_path = "./aida_train_instanceof.jsonl"
from datasets import Dataset

original_data_aida = []
with open(file_path, "r") as file:
    for line in file:
        # Load each line as a JSON object
        data_line = json.loads(line)
        
        # additional_question = "<additional> "
        # try:
        #     for related in data_line["most_related"]:
        #         length = len(related)
        #         if len(related) == 4:
        #             additional_question += related[2]
        #             additional_question += " "
        # except:
        #     None
        # additional_question += "</additional>"
        # data_line["context"] += additional_question
        original_data_aida.append(data_line)
print(len(original_data_aida))

18448


In [5]:
import random
random.shuffle(original_data_aida)
#concat = original_data_aida
concat =original_data_aida[:int(len(original_data_aida)* 0.7)] + original_data


random.shuffle(concat)
#print(len(concat))
train_original = concat[: int(len(concat) * 0.95)]
eval_original = concat[int(len(concat) * 0.95) : ]
dataset = Dataset.from_dict(
    {
        "context": [item["question"] for item in train_original],
        "question": [item["context"] for item in train_original],
        "answers": [item["answers"] for item in train_original],
    }
)
dataset_ev = Dataset.from_dict(
    {
        "context": [item["question"] for item in eval_original],
        "question": [item["context"] for item in eval_original],
        "answers": [item["answers"] for item in eval_original],
    }
)

In [6]:
def get_correct_alignement(context, answer):
    """Some original examples in SQuAD have indices wrong by 1 or 2 character. We test and fix this here."""
    gold_text = answer["text"][0]
    
    start_idx = context.find(gold_text)
    end_idx = context.find("</ec>", start_idx)
    return start_idx, end_idx
    # if context[start_idx:end_idx] == gold_text:
    #     return start_idx, end_idx       # When the gold label position is good
    # elif context[start_idx-1:end_idx-1] == gold_text:
    #     return start_idx-1, end_idx-1   # When the gold label is off by one character
    # elif context[start_idx-2:end_idx-2] == gold_text:
    #     return start_idx-2, end_idx-2   # When the gold label is off by two character
    # else:
    #     raise ValueError()


# Tokenize our training dataset
def convert_to_features(example):
    try:
        # Tokenize contexts and questions (as pairs of inputs)

        input_pairs = [example["question"], example["context"]]
        encodings = tokenizer.encode_plus(
            input_pairs, pad_to_max_length=True, truncation="only_second"
        )
        context_encodings = tokenizer.encode_plus(example["context"])

        # Compute start and end tokens for labels using Transformers's fast tokenizers alignement methodes.
        # this will give us the position of answer span in the context text

        start_idx, end_idx = get_correct_alignement(
            example["context"], example["answers"]
        )
        if end_idx != -1 and start_idx != -1:
            try:
                start_positions_context = context_encodings.char_to_token(start_idx)
                end_positions_context = context_encodings.char_to_token(end_idx)

                # here we will compute the start and end position of the answer in the whole example
                # as the example is encoded like this <s> question</s></s> context</s>
                # and we know the postion of the answer in the context
                # we can just find out the index of the sep token and then add that to position + 1 (+1 because there are two sep tokens)
                # this will give us the position of the answer span in whole example
                sep_idx = encodings["input_ids"].index(tokenizer.sep_token_id)
                start_positions = start_positions_context + sep_idx
                end_positions = end_positions_context + sep_idx + 1
                #print(tokenizer.decode
                #(encodings["input_ids"][start_positions:end_positions]))
                encodings.update(
                    {
                        "start_positions": start_positions,
                        "end_positions": end_positions,
                        "attention_mask": encodings["attention_mask"],
                    }
                )
                return encodings
            except Exception as e:
                print(e)
        else:
            print("qu")
            encodings.update(
                {
                    "start_positions": start_idx,
                    "end_positions": end_idx,
                    "attention_mask": encodings["attention_mask"],
                    "is_nil": 1 if example["answers"]["text"][0] == "Not In Candidates" else 0
                }
            )
            return encodings
    except Exception as e:
        print("errors", e)

In [ ]:
train_ds = dataset.map(convert_to_features)
eval_ds = dataset_ev.map(convert_to_features) 

In [ ]:
import joblib
import datasets

In [ ]:
joblib.dump( train_ds, './dumps/aidaTrainIOF.dump',)
joblib.dump( eval_ds, './dumps/aidaEvalIOF.dump',)

In [ ]:
train_ds = joblib.load('./dumps/aidaTrainNERBigBird.dump')
eval_ds = joblib.load('./dumps/aidaEvalNERBigBird.dump')

In [16]:
wandb.init()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: wandb version 0.16.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [8]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator

import wandb
torch.cuda.set_device(0)
torch.cuda.current_device()

#wandb.init(project="tesi", name="robertaAidaLargeNERNIL")  # args = TrainingArguments(
#     f"longformer-finetuned-squad",
#     evaluation_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=8,
#     num_train_epochs=3,
#     weight_decay=0.01,
# )

args = TrainingArguments(
    "robertaLargefNil",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    report_to="wandb",
    load_best_model_at_end=False,
    logging_strategy="steps",
    logging_steps=500,
    eval_steps=500,
    save_steps=500,
    save_total_limit=1,
    learning_rate=3e-5,
    per_device_eval_batch_size=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=100,
    num_train_epochs=14,
    weight_decay=0.02,
    fp16=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Using amp fp16 backend


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: answers, context, question.
***** Running training *****
  Num examples = 19809
  Num Epochs = 14
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 100
  Gradient Accumulation steps = 100
  Total optimization steps = 2772
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: agazzi-ruben99 (use `wandb login --relogin` to force relogin)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: wandb version 0.16.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch,Training Loss,Validation Loss
0,No log,0.386881
1,No log,0.337372
2,0.466700,0.329125
3,0.466700,0.356657
4,0.466700,0.387810


/opt/miniconda3/envs/extend/lib/python3.9/site-packages/transformers/trainer.py:1355: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  nn.utils.clip_grad_norm_(
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: answers, context, question.
***** Running Evaluation *****
  Num examples = 1043
  Batch size = 1
Saving model checkpoint to robertaLargefNil/checkpoint-198
Configuration saved in robertaLargefNil/checkpoint-198/config.json
Model weights saved in robertaLargefNil/checkpoint-198/pytorch_model.bin
tokenizer config file saved in robertaLargefNil/checkpoint-198/tokenizer_config.json
Special tokens file saved in robertaLargefNil/chec

In [ ]:
trainer.save_model('./RoBertAAida')

In [ ]:
torch.cuda.empty_cache()

In [ ]:
dataset[0]

## Prediction and evaluation

In [2]:
def check_brackets(input_string):
    stack = []
    brackets = {"(": ")", "{": "}", "[": "]"}
    for char in input_string:
        if char in brackets.keys():
            stack.append(char)
        elif char in brackets.values():
            if not stack or brackets[stack.pop()] != char:
                return False
    return not stack


def process_answer(answer, candidates):
    if answer == "Not In Candidates":
        return answer
    else:
        modified_answer = answer.split("</ec>")[0]
        modified_answer = modified_answer.split(": instance of ")[0]
        modified_answer = modified_answer.split(": instance")[0]
        modified_answer = modified_answer.replace("<s>", "")
        modified_answer = modified_answer.replace("</ec", "")
        modified_answer = modified_answer.replace("</s>", "")
        modified_answer = modified_answer.replace("<s", "")
        modified_answer = modified_answer.replace("</", "")
        modified_answer = modified_answer.replace(" ec", "")
        modified_answer = modified_answer.replace(">", "")
        modified_answer = modified_answer.strip()
        if not check_brackets(modified_answer):
            modified_answer = modified_answer.replace("(", "")
            modified_answer = modified_answer.replace(")", "")
        if modified_answer == "Not In Candidates":
            modified_answer = "NIL"
        if modified_answer == "":
            modified_answer = "Not In Candidates"
        if modified_answer not in candidates:
            modified_answer = "Not In Candidates"
        return modified_answer


process_answer("> Alzenau </", ["Alzenau"])

'Alzenau'

In [3]:
import gc
import torch.nn.functional as F


def make_prediction(data_entry, nil_prediction):
    with torch.no_grad():
        question = data_entry["input"]
        context = ""
        # if nil_prediction:
        #     context += " Not In Candidates </ec> "
        index = 0
        added = False
        for item in data_entry["candidates"]:
            context += item + f" </ec> "
            index += 1
            if index == 1 and nil_prediction:
                context += " Not In Candidates : instance of Unknown </ec> "
                added = True
        if not added and nil_prediction:
            context = " Not In Candidates : instance of Unknown </ec> " + context
        input_pairs = [question, context]

        encodings = tokenizer.encode_plus(
            input_pairs, return_tensors="pt", truncation="only_second"
        ).to("cuda")
        start_scores, end_scores = model(
            encodings["input_ids"], attention_mask=encodings["attention_mask"]
        )
        start_scores = F.softmax(start_scores, dim=1)
        end_scores = F.softmax(end_scores, dim=1)
        # start_scores.to("cpu")
        # end_scores.to("cpu")
        start = torch.argmax(
            start_scores
        )  # Get the most likely beginning of answer with the argmax of the score
        end = (
            torch.argmax(end_scores) + 1
        )  # Get the most likely end of answer with the argmax of the score
        mean_score = 0
        try:
            mean_score = end_scores[0][end.item()] + start_scores[0][start.item()]
        except:
            None
        answer_tokens = encodings["input_ids"][0, start.item() : end.item() + 1]
        answer = ""

        answer = process_answer(tokenizer.decode(answer_tokens), context)
        classified = 0
        if (mean_score < 0.9) and nil_prediction:
            answer = "Not In Candidates"
        else:
            if answer == "":
                answer = "Not In Candidates"
        del encodings
        del end_scores
        del start_scores
        del start
        del end
        score = float(mean_score)
        gc.collect()
        torch.cuda.empty_cache()
        return {
            "correct": data_entry["output"],
            "non_processed": tokenizer.decode(answer_tokens),
            "predicted": answer,
            "input_phrase": question,
            "scores": score,
            "candidates": context,
        }

In [ ]:
import wandb
wandb.init()

In [5]:
from tqdm import tqdm
import json
import torch

results = []
file_path = "./aida-test-kilt-instanceof.jsonl"
dataset = []
with open(file_path, "r") as file:
    for line in file:
        # Load each line as a JSON object
        data_line = json.loads(line)

        dataset.append(data_line)

for item in tqdm(dataset):
    try:
        pred = make_prediction(item, False)
        results.append(pred)
    except Exception as e:
        print(e)

100%|██████████| 4485/4485 [11:24<00:00,  6.55it/s]


In [7]:
correct

3968

In [6]:
correct = 0
correct_trace = []
wrong = []
correct_nil = 0
wrong_nil = 0
full_data = []
for result in tqdm(results):
  processed = result['predicted']
  if processed == result['correct'][0]['answer']:
    correct += 1
    correct_trace.append(result)
    if processed == 'Not In Candidates':
        correct_nil += 1
  else:
    if result['correct'][0]['answer'] == 'Not In Candidates':
        wrong_nil += 1
    wrong.append(result)
  full_data.append({'score': result['scores'], 'nil': int(result['correct'][0]['answer'] == 'Not In Candidates')} )

100%|██████████| 4485/4485 [00:00<00:00, 814383.89it/s]


In [7]:
print(f"accuracy of the model is {correct/len(results)}")
print(f"accuracy in nil prediction is: {correct_nil/(correct_nil+wrong_nil)}")

accuracy of the model is 0.8923076923076924


ZeroDivisionError: division by zero

In [33]:
print(f"accuracy of the model is {correct/len(results)}")
print(f"accuracy in nil prediction is: {correct_nil/(correct_nil+wrong_nil)}")

accuracy of the model is 0.836890243902439
accuracy in nil prediction is: 0.5935483870967742


-  Base: 65.9%
-  Large: 73.21%(enriched) - 74.65% (standard)
-  Large EN: 76.66%
-  large NER: 75.5%
-  Base-NER: 66.35%

## MSNBC large
-  No EN: 91.92%
-  No Instance Of: 92.83% - Yes: 93.44%
-  EN: 92.8
-  Large NER: 92.83%

In [ ]:
len(wrong)

In [7]:
import pandas as pd
full_df = pd.DataFrame(full_data)
df = pd.DataFrame(wrong)
df_c = pd.DataFrame(correct_trace)

In [115]:
df.iloc[0]['candidates']

'Iraq War : instance of military offensive </ec> Iraq national football team : instance of national association football team </ec> Anglo-Iraqi War : instance of war </ec> Iraq Football Association : instance of association football federation </ec> Iraqi Army : instance of Unknown </ec> Iraq at the 2004 Summer Olympics : instance of Olympic delegation </ec> Iraq at the 2008 Summer Olympics : instance of Olympic delegation </ec> Iraq at the 1960 Summer Olympics : instance of Olympic delegation </ec> Iraq prison abuse scandals : instance of scandal </ec> Iraq at the Asian Games : instance of nation at sport competition </ec> Iraq national basketball team : instance of Unknown </ec> History of Iraq (2003–11) : instance of Unknown </ec> Iraq at the 2006 Asian Games : instance of nation at sport competition </ec> Iraqi Premier League : instance of Unknown </ec> Iraq national futsal team : instance of national futsal team </ec> Afro-Iraqi : instance of Unknown </ec> Iraq–Turkey relations : 

In [8]:
n = 20
offset=1
df.iloc[offset *n : offset*n + n]

,correct,non_processed,predicted,input_phrase,scores,candidates
20,"[{'answer': 'Abdelbaset al-Megrahi', 'provenan...",,Not In Candidates,is that the judges may not punish Abu Talib un...,0.998908,
21,"[{'answer': 'New York', 'provenance': [{'title...",> New York City : instance of city in the Unit...,New York City,N Y Times News Service LR QC QL GROSSE POINTE ...,0.999954,New York : instance of Wikimedia disambiguatio...
22,[{'answer': 'Michigan Department of Natural Re...,</s>Minnesota Department of Natural Resources ...,Minnesota Department of Natural Resources,injured since the hunting season opened on Nov...,0.999875,Minnesota Department of Natural Resources : in...
23,"[{'answer': 'Joel Grossman', 'provenance': [{'...",,Not In Candidates,Welcome to this hour of VOA News Now I m Erin ...,0.925782,
24,"[{'answer': 'Florida Legislature', 'provenance...",</s>Legislature : instance of Unknown </ec,Legislature,we have had a Supreme Court challenge over pre...,0.999995,Legislature : instance of Unknown </ec> Federa...
25,"[{'answer': 'Netzarim (settlement)', 'provenan...",,Not In Candidates,the Israeli army began bulldozing Palestinian ...,0.978799,
26,"[{'answer': 'State Street Corporation', 'prove...",</s>,Not In Candidates,shareholders with 55 percent of the merged com...,1.005916,
27,"[{'answer': 'Time Inc.', 'provenance': [{'titl...",</s>Time (magazine) : instance of news magazin...,Time (magazine),company you re going to be a happy camper That...,0.999403,Time (magazine) : instance of news magazine </...
28,"[{'answer': 'Warner Communications', 'provenan...",> Warner Bros. : instance of entertainment com...,Warner Bros.,re going to be a happy camper That view is wid...,0.999943,Warner Music Group : instance of record compan...
29,"[{'answer': 'Janus Capital Group', 'provenance...",,Not In Candidates,McNamee of Integral Capital which sold its 404...,0.999672,


In [44]:
df.to_csv("./errorsNil.csv")
df_c.to_csv("./correctNil.csv")